# Sequence-to-sequence with attention for microbial growth prediction

This notebook contains all functions needed to implement Seq2Seq+Attn model using Tensorflow/Keras.

**The data is property of PhageLab Chile SpA, and hence it is not publicly available.** 

The models can be trained on any growth curve.

Correspondence: lleon@pht.cl

In [ ]:
!pip install sktime -q

In [ ]:
import numpy as np
import random

import tensorflow as tf
from tensorflow.keras import activations, regularizers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import *
from tensorflow.keras import backend as K
from tensorflow.keras.utils import get_custom_objects, pad_sequences
from tensorflow.keras.optimizers import Adamax

from scipy import stats
from sklearn.preprocessing import QuantileTransformer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_percentage_error
from sktime.performance_metrics.forecasting import mean_squared_scaled_error, median_squared_scaled_error, mean_absolute_scaled_error, median_absolute_scaled_error, mean_squared_error, mean_absolute_error, MeanAbsolutePercentageError

np.random.seed(1)
tf.random.set_seed(1)
random.seed(1)

# Model implementation and other functions

In [ ]:
def get_tstudent_ci(samples, n=100, l=0.95):
    sample_mean = np.mean(samples, axis=1)
    sample_std = np.std(samples, axis=1)
    alpha = 1-l
    t_critical = stats.t.ppf(1-alpha/2, df=3)
    sem = sample_std/np.sqrt(n)
    margin_of_error = t_critical*sem
    lower_bound = np.squeeze(sample_mean-margin_of_error).T
    upper_bound = np.squeeze(sample_mean+margin_of_error).T
    return (lower_bound, upper_bound)

class StackedGRU(Model):
    def __init__(self, latent_dim, num_rnn=1):
        super(StackedGRU, self).__init__()
        self.latent_dim = latent_dim
        self.num_rnn = num_rnn

        self.init_rnn = GRU(latent_dim, return_sequences=True, name='init_rnn')
        if self.num_rnn:
            self.stacked_rnn = Sequential(name="stacked_rnn")
            for i in range(num_rnn):
                self.stacked_rnn.add(GRU(latent_dim, return_sequences=True))
        
        self.out_rnn = GRU(latent_dim, return_sequences=True, return_state=True, name='out_rnn')

    def call(self, inputs, initial_state=None):
        x = self.init_rnn(inputs, initial_state=initial_state)
        if self.num_rnn>0:
            x = self.stacked_rnn(x)
        x = self.out_rnn(x)
        return x
    
class DenseD(Model):
    def __init__(self, out_dim, name, num_dense=0, extra_dense_dim=16, activation='linear'):
        super(DenseD, self).__init__()
        self.out_dim = out_dim
        self.num_dense = num_dense
        self.extra_dense_dim = extra_dense_dim
        self.activation = activation

        if num_dense > 0:
            self.extra_dense = Sequential(name="extra_dense")
            for i in range(num_dense):
                self.extra_dense.add(TimeDistributed(Dense(extra_dense_dim)))
                self.extra_dense.add(Dropout(0.2))
                
        self.dense_out = TimeDistributed(Dense(self.out_dim, activation=self.activation))
        self.correction = TimeDistributed(Dense(self.out_dim))

    def call(self, inputs):
        x = inputs
        if self.num_dense > 0:
            x = self.extra_dense(x)
        x = self.dense_out(x)
        corr = self.correction(inputs)
        return x+corr

def seq2seq_with_attention(
    input_shape, 
    dec_input_shape, 
    output_sequence_length, 
    latent_dim, 
    dropout_rate=0.2,
    mask_value=-10
):
    """
    This function implements the seq2seq model with attention to predict growth curves.
    
    """
    
    # GRU Encoder part
    encoder_inputs = Input(shape=input_shape, name="encoder_inputs")    
    encoder_inputs_masked = Masking(mask_value=mask_value)(encoder_inputs)
    
    # GRU block
    encoder_gru = StackedGRU(latent_dim, 3)
    encoder_outputs, enc_gru_state = encoder_gru(encoder_inputs_masked)
    encoder_states = [enc_gru_state]
    
    # GRU Decoder part
    decoder_inputs = Input(shape=(None, dec_input_shape[1]), name="decoder_inputs")
    decoder_inputs_masked = Masking(mask_value=-10)(decoder_inputs)
    decoder_state_input = Input(shape=(latent_dim,), name="decoder_state_input")
    decoder_states_inputs = [decoder_state_input]

    decoder_gru = GRU(
        latent_dim, 
        return_sequences=True, 
        return_state=True, 
        name="dec_gru"
    )
    decoder_gru_output, _ = decoder_gru(decoder_inputs_masked, initial_state=encoder_states)
    # Student's parameters mu, sigma
    decoder_dense_mean = DenseD(1, num_dense=0, name="dec_mean", activation='sigmoid')
    decoder_dense_log_var = DenseD(1, num_dense=0, name="dec_var", activation='linear')

    # Attention block
    attention = Attention(name="attention", use_scale=True, dropout=0.2, score_mode="dot")
    # the context
    context_vector = attention([decoder_gru_output, encoder_outputs], use_causal_mask=True)
    decoder_combined_context = Concatenate(axis=-1, name="dec_comb_context")([decoder_gru_output, context_vector])
    dropout_layer = Dropout(0.2)
    decoder_combined_context = dropout_layer(decoder_combined_context)
    
    # apply functions for mu and sigma
    decoder_mean = decoder_dense_mean(decoder_combined_context)
    decoder_log_var = decoder_dense_log_var(decoder_combined_context)
    
    # concatenate the results into a single output
    decoder_outputs = Concatenate(axis=-1, name="dec_outputs")([decoder_mean, decoder_log_var])
    
    # The full model
    full_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    
    # Encoder model
    encoder_model = Model(encoder_inputs, [encoder_outputs]+encoder_states)
    
    # Decoder model (single prediction)
    # Here we reuse the decoder layer and the attention
    decoder_inputs_single = Input(shape=(1, dec_input_shape[1]), name="dec_input_single")
    decoder_inputs_single_masked = Masking(mask_value=-10)(decoder_inputs_single)
    encoder_outputs_input = Input(shape=(None, latent_dim), name="encoder_outputs_single")

    decoder_gru_output_single, dec_gru_state_single = decoder_gru(decoder_inputs_single_masked, initial_state=decoder_states_inputs)
    context_vector_single = attention([decoder_gru_output_single, encoder_outputs_input], use_causal_mask=True)
    decoder_combined_context_single = Concatenate(axis=-1)([decoder_gru_output_single, context_vector_single])
    
    decoder_mean_single = decoder_dense_mean(decoder_combined_context_single)
    decoder_log_var_single = decoder_dense_log_var(decoder_combined_context_single)
    decoder_outputs_single = Concatenate(axis=-1, name="decoder_outputs_single")([decoder_mean_single, decoder_log_var_single])
    
    decoder_states = [dec_gru_state_single]
    decoder_model = Model([decoder_inputs_single, encoder_outputs_input] + decoder_states_inputs, [decoder_outputs_single] + decoder_states)
    
    return full_model, encoder_model, decoder_model

def simple_seq2seq(input_shape, dec_input_shape, output_sequence_length, latent_dim, dropout_rate=0.2):
    """
    This function implements the simple seq2seq model to predict growth curves.
    
    """
    # GRU Encoder part
    encoder_inputs = Input(shape=input_shape, name="encoder_inputs")
    encoder_inputs_masked = Masking(mask_value=-10)(encoder_inputs)
    
    # GRU block
    encoder_gru = StackedGRU(latent_dim, 3)
    encoder_outputs, enc_gru_state = encoder_gru(encoder_inputs_masked)
    encoder_states = [enc_gru_state]
    
    # GRU Decoder part
    decoder_inputs = Input(shape=(None, dec_input_shape[1]), name="decoder_inputs")
    decoder_inputs_masked = Masking(mask_value=-10)(decoder_inputs)
    decoder_state_input = Input(shape=(latent_dim,), name="decoder_state_input")
    decoder_states_inputs = [decoder_state_input]

    decoder_gru = GRU(
        latent_dim, 
        return_sequences=True, 
        return_state=True, 
        name="dec_gru"
    )
    decoder_gru_output, _ = decoder_gru(decoder_inputs_masked, initial_state=encoder_states)

    # Student's parameters mu, sigma
    decoder_dense_mean = DenseD(1, num_dense=0, name="dec_mean", activation='sigmoid')
    decoder_dense_log_var = DenseD(1, num_dense=0, name="dec_var", activation='linear')
    
    # apply functions for mu and sigma
    decoder_mean = decoder_dense_mean(decoder_gru_output)
    decoder_log_var = decoder_dense_log_var(decoder_gru_output)
    
    # concatenate the results into a single output
    decoder_outputs = Concatenate(axis=-1, name="dec_outputs")([decoder_mean, decoder_log_var])
    
    # The full model
    full_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    
    # Encoder model
    encoder_model = Model(encoder_inputs, [encoder_outputs]+encoder_states)
    
    # Decoder model (single prediction)
    # Here we reuse the decoder layer
    decoder_inputs_single = Input(shape=(1, dec_input_shape[1]), name="dec_input_single")
    decoder_inputs_single_masked = Masking(mask_value=-10)(decoder_inputs_single)
    encoder_outputs_input = Input(shape=(None, latent_dim), name="encoder_outputs_single")

    decoder_gru_output_single, dec_gru_state_single = decoder_gru(decoder_inputs_single_masked, initial_state=decoder_states_inputs)
    
    decoder_mean_single = decoder_dense_mean(decoder_gru_output_single)
    decoder_log_var_single = decoder_dense_log_var(decoder_gru_output_single)
    decoder_outputs_single = Concatenate(axis=-1, name="decoder_outputs_single")([decoder_mean_single, decoder_log_var_single])
    
    decoder_states = [dec_gru_state_single]
    decoder_model = Model([decoder_inputs_single, encoder_outputs_input] + decoder_states_inputs, [decoder_outputs_single] + decoder_states)
    
    return full_model, encoder_model, decoder_model

def forecast_probabilistic_gru(
    encoder_model, 
    decoder_model, 
    input_sequence, 
    output_sequence_length, 
    n_samples,
    n_features, 
    latent_dim, 
    n_realizations=100,
    start_token=0.5,
    t_student=False
):
    encoder_outputs, encoder_states = encoder_model.predict(input_sequence, verbose=0)
    encoder_states = [encoder_states]

    target_seq = start_token*np.ones((n_samples, 1, n_features))

    decoded_sequence = []

    for _ in range(output_sequence_length):
        output_tokens, decoder_states = decoder_model.predict(
            [target_seq, encoder_outputs] + encoder_states,
            verbose=0
        )

        mean = output_tokens[..., 0]
        log_var = output_tokens[..., 1]
        var = np.exp(log_var)

        if t_student:
            samples = [stats.t.rvs(3, loc=mean, scale=np.sqrt(var)) for _ in range(n_realizations)]
            samples = np.array(samples)
        else:
            samples = [np.random.normal(mean, np.sqrt(var)) for _ in range(n_realizations)]
            samples = np.array(samples)

        decoded_sequence.append(samples)

        target_seq = np.zeros((n_samples, 1, n_features))
        target_seq[:, 0, 0] = np.squeeze(mean)

        encoder_states = [decoder_states]

    return np.array(decoded_sequence)

def student_t_nll(y_true, y_pred):
    eps = 1e-8
    mu = y_pred[..., 0]
    log_sigma = y_pred[..., 1]
    sigma = tf.exp(log_sigma)+eps  # Ensure sigma is positive    
    nu = 3.0 # dof
    term1 = y_pred[..., 1]  # Log of the scale
    term2 = (nu+1.0)/2.0*K.log(1+(1/nu)*K.square((y_true[..., 0]-mu)/sigma))
    nll = term1+term2
    return K.mean(nll)

def create_test_sequences(data, input_seq_len, output_seq_len, start_token=0.0):
    n_samples, time, n_features = data.shape
    X_encoder, X_decoder, y = [], [], []

    for sample in range(n_samples):
        for i in range(time - input_seq_len - output_seq_len + 1):
            input_seq = data[sample, i:i+input_seq_len]
            output_seq = data[sample, i+input_seq_len:i+input_seq_len+output_seq_len]
            X_encoder.append(input_seq)
            X_decoder.append(output_seq)
            y.append(output_seq)

    # Convert to numpy arrays
    X_encoder = np.array(X_encoder)
    X_decoder = np.array(X_decoder)
    y = np.array(y)

    # Shift X_decoder
    X_decoder = np.concatenate([start_token*np.ones((X_decoder.shape[0], 1, n_features)), X_decoder[:, :-1, :]], axis=1)

    return X_encoder, X_decoder, y

def variable_length_per_sample(
    time_series_data, 
    padding_value=-10,
    start_token=-100,
    lowl=36,
    highl=48,
    use_max_samps=False,
    max_samps=5000,
    sample_weight=None,
    diff=False
):

    # Determine the number of samples and length of each series
    num_samples, series_length, num_features = time_series_data.shape
    data = np.squeeze(time_series_data)

    # Lists to hold variable-length sequences
    encoder_input_data_list = []
    decoder_input_data_list = []
    decoder_target_data_list = []

    split_index = [12, 36, 42, 48, 96]
    for series in data:
        for si in split_index:
            # Encoder input sequence
            encoder_input = series[:si]
            if diff:
                encoder_diff = np.zeros_like(encoder_input)
                encoder_diff[1:] = np.diff(encoder_input)
                encoder_input = np.concatenate((encoder_input[:, None], encoder_diff[:, None]), axis=1)
            encoder_input_data_list.append(encoder_input)

            # Decoder input sequence
            decoder_input = np.concatenate(([start_token], series[si:-1]))
            decoder_input_data_list.append(decoder_input)

            # Decoder target sequence is the decoder input shifted by one
            decoder_target = series[si:]
            decoder_target_data_list.append(decoder_target)

    # Pad sequences to the maximum length in their respective lists
    encoder_input_data = pad_sequences(
        encoder_input_data_list, 
        padding='post', 
        value=padding_value, 
        dtype=data.dtype
    )
    decoder_input_data = pad_sequences(
        decoder_input_data_list, 
        padding='post', 
        value=padding_value, 
        dtype=data.dtype
    )
    decoder_target_data = pad_sequences(
        decoder_target_data_list, 
        padding='post', 
        value=padding_value, 
        dtype=data.dtype
    )

    # shuffle
    idx = np.arange(len(encoder_input_data))
    np.random.shuffle(idx)
    encoder_input_data = encoder_input_data[idx]
    decoder_input_data = decoder_input_data[idx]
    decoder_target_data = decoder_target_data[idx]
    if sample_weight is not None:
        sample_weight = np.repeat(sample_weight, len(split_index))
        sample_weight = sample_weight[idx]
    

    # masks
    encoder_mask = encoder_input_data!=padding_value
    decoder_mask = decoder_input_data!=padding_value
    target_mask = decoder_target_data!=padding_value
    
    if not diff:
        encoder_input_data = np.expand_dims(encoder_input_data, -1)
    decoder_input_data = np.expand_dims(decoder_input_data, -1)
    decoder_target_data = np.expand_dims(decoder_target_data, -1)
        

    if use_max_samps:
        return (
            encoder_input_data[:max_samps], 
            decoder_input_data[:max_samps], 
            decoder_target_data[:max_samps],
            encoder_mask[:max_samps],
            decoder_mask[:max_samps],
            target_mask[:max_samps]
        )
    
    if sample_weight is not None:
        return (
        encoder_input_data, 
        decoder_input_data, 
        decoder_target_data,
        encoder_mask,
        decoder_mask,
        target_mask,
        sample_weight
    )
    
    return (
        encoder_input_data, 
        decoder_input_data, 
        decoder_target_data,
        encoder_mask,
        decoder_mask,
        target_mask
    )

# Random input data
Due to the privacy of the data generated by PhageLab Chile SpA, we create random data for the purpose of running this script. 

In [ ]:
N = 5000
L = 109
feats = 1
shape = (N, L, feats)
X = np.random.normal(size=shape)
categories = np.random.randint(0, 4, size=N)

# Training loop

In [ ]:
# Example usage
num_features = 1
latent_dim = 128  
lr = 1e-3
epochs = 5
t_student = True
low_lim = 12
up_lim = 96
start_token = -100
n_folds = 3

skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=1)

In [ ]:
metrics_per_fold = {}
scalers = {}
test_input_length = 36
total_length = 109

for i, (train_index, test_index) in enumerate(skf.split(X, categories)):
    fold_name = f"fold_{i}"
    
    X_train = X[train_index]
    X_test = X[test_index]
    
    scaler = QuantileTransformer(output_distribution="normal", n_quantiles=1000).fit(X_train.flatten()[:, None])
    X_train = np.array(list(map(scaler.transform, X_train)))
    X_test = np.array(list(map(scaler.transform, X_test)))
    
    # Create sequences    
    X_encoder_train, X_decoder_train, y_train, encoder_mask_train, decoder_mask_train, y_mask_train = variable_length_per_sample(X_train, lowl=low_lim, highl=up_lim)
    
    X_encoder_test, X_decoder_test, y_test = create_test_sequences(
        X_test, 
        test_input_length, 
        total_length-test_input_length,
        start_token=start_token
    )
    # pad encoder_test to make it compatible with the model
    X_encoder_test = np.pad(
        X_encoder_test, 
        pad_width=((0,0),(0,up_lim-test_input_length),(0,0)), 
        mode='constant', 
        constant_values=-10
    )
    
    input_shape_masked = (X_encoder_train.shape[1], num_features)
    dec_input_shape = (X_encoder_train.shape[1], num_features)
    output_shape_masked = X_decoder_train.shape[1]
    
    # load architecture
    model, encoder_model, decoder_model = seq2seq_with_attention(
        input_shape_masked, 
        dec_input_shape,
        output_shape_masked, 
        latent_dim=latent_dim
    )

    if t_student:
        loss = student_t_nll
    else:
        loss = gaussian_nll_loss
    opt = Adamax(
        learning_rate=lr,
    )

    model.compile(optimizer=opt, loss=loss)
    
    # Train the model
    model.fit(
        [X_encoder_train, X_decoder_train], 
        y_train, 
        epochs=epochs, 
        batch_size=128,
    )
    
    # Eval
    # Perform the forecast with probabilistic output and attention   
    samples = forecast_probabilistic_gru(
        encoder_model, 
        decoder_model, 
        X_encoder_test, 
        X_decoder_test.shape[1],
        X_encoder_test.shape[0],
        num_features, 
        latent_dim,
        start_token=start_token,
        t_student=t_student
    )

    # For metric estimation, we need to undo the scaling
    X_decoder_test = scaler.inverse_transform(X_decoder_test.flatten()[:, None]).reshape(X_decoder_test.shape)
    X_encoder_test = scaler.inverse_transform(X_encoder_test.flatten()[:, None]).reshape(X_encoder_test.shape)
    samples = scaler.inverse_transform(samples.flatten()[:, None]).reshape(samples.shape)
    
    mean_prediction = np.squeeze(np.mean(samples, axis=1)).T
    y_true = scaler.inverse_transform(y_test.flatten()[:, None]).reshape(y_test.shape)
    y_true = np.squeeze(y_true)
    ci_level = get_tstudent_ci(samples)
    lower_bound, upper_bound = ci_level
    
    smape_cls = MeanAbsolutePercentageError(symmetric=True)
    metrics_per_fold[fold_name] = {
        'MAE': mean_absolute_error(y_true, mean_prediction),
        'MSE': mean_squared_error(y_true, mean_prediction),
        'SMAPE': smape_cls(y_true, mean_prediction),
        'MASE': mean_absolute_scaled_error(y_true, mean_prediction, y_train=np.squeeze(X_decoder_test)),
        'RMSSE': mean_squared_scaled_error(y_true, mean_prediction, y_train=np.squeeze(X_decoder_test)),
    }
    
    scalers[fold_name] = minmax_scaler